In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import curve_fit

## References

In [ ]:
def int_func(x):
    if x < 3:
        return x**2
    else:
        return 2 + x**2

In [ ]:
def func(x, a):
    # int_func = lambda y: y**2

    res = np.asarray([a*quad(int_func, 0, t)[0] for t in x])
    return res

In [ ]:
curve_fit(func, [1,2,3], [6,3,4])

## Fungsi

In [2]:
from lib.uem.metrices import Compute
from lib.uem.biomass import Biomassa
from lib.helpers import price_function, source_data, score_csc_compute
from lib.uem.feeding_rate import feeding_rate

f_uia, f_o2, f_temp, temperature = source_data(
    path = "data/growth_full.csv",
    temp_suitable_min = 25,
    temp_suitable_max = 33,
    temp_optimal_min = 28,
    temp_optimal_max = 32,
    do_suitable_min = 4,
    do_suitable_max = 10,
    do_optimal_min = 6,
    do_optimal_max = 9,
    ua_suitable_min = 0.00,
    ua_suitable_max = 0.16,
    ua_optimal_min = 0.00,
    ua_optimal_max = 0.06,
)


csc_suitable_min = 0.00
csc_suitable_max = 5
csc_optimal_min = 0.00
csc_optimal_max = 3

In [3]:
from lib.uem.integrate_fr import Fungsi

### Testing using dummy function

In [ ]:
def dummy_func(T, alpha):
    res = np.asarray([alpha * Fungsi(0, t, f_uia, f_o2, f_temp, 0, 0).get_integral()[0] for t in T])
    return res

In [ ]:
curve_fit(dummy_func, [1,2,3], [10, 30, 20])

In [8]:
def single_wt1(t, sr, m, alpha):
    # print(t)
    try:
        feedRate = feeding_rate(0, float(temperature[temperature["DOC"]==t]["Temp"]), 0)
    except  :
        feedRate = 0
   
    if t == 0:
        score_csc = score_csc_compute(0/1000, 0.01, csc_suitable_min, csc_suitable_max, csc_optimal_max)
        obj = Biomassa(0, t, 40, 0.05, alpha, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, score_csc, feedRate, 0, final_doc=120)
        wt = obj.wt()
        biomass = obj.biomassa()
        cum_integral = obj._fr()
        return wt, biomass, cum_integral
    else:
        wt_1, bio_1, cum_1 = single_wt1(t-1, sr, m, alpha)
        score_csc = score_csc_compute(bio_1/1000, 1000, csc_suitable_min, csc_suitable_max, csc_optimal_max)
        obj = Biomassa(t-1, t, 40, 0.05, alpha, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, score_csc, feedRate, cum_1, 120)
        wt = obj.wt()
        biomass = obj.biomassa()
        cum_integral = cum_1 + obj._fr()

        return wt, biomass, cum_integral

def single_wt(T, sr, m, alpha):
    
    weight = []
    biomassa = []
    cum_integral = 0
    for i in range(int(T) + 1):
        try:
            feedRate = feeding_rate(0, float(temperature[temperature["DOC"]==i-1]["Temp"]), 0)
        except  :
            feedRate = 0

        if i == 0:
            score_csc = score_csc_compute(0/1000, 0.01, csc_suitable_min, csc_suitable_max, csc_optimal_max)
            obj = Biomassa(0, i, 40, 0.05, alpha, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, score_csc, feedRate, cum_integral, final_doc=120)
            wt = obj.wt()
            biomass = obj.biomassa()
            cum_integral = cum_integral + obj._fr()
        else:
            score_csc = score_csc_compute(biomassa[-1]/1000, 1000, csc_suitable_min, csc_suitable_max, csc_optimal_max)
            obj = Biomassa(i-1, i, 40, 0.05, alpha, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, score_csc, feedRate, cum_integral, 120)
            wt = obj.wt()
            biomass = obj.biomassa()
            cum_integral = cum_integral + obj._fr()
        

        weight.append(wt)
        biomassa.append(biomass)

    return wt

def multi_wt(T, alpha):
    sr = 0.92
    m = -np.log(sr)/T[-1]
    # print(T)
    # res = np.asarray([single_wt1(t, sr, m, alpha)[0] for t in T])
    res = np.asarray([single_wt(t, sr, m, alpha) for t in T])
    return res

In [10]:
curve_fit(multi_wt, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0.0145, 0.028, 0.0415, 0.055, 0.0685, 0.082, 0.0955, 0.109, 0.1225, 0.2025])

(array([4.38622441e-05]), array([[1.69381199e-11]]))

In [8]:
import pandas as pd

df = pd.read_csv("data/growth_full.csv", sep=";")

In [17]:
curve_fit(multi_wt, df["DOC"][:20].tolist(), df["ABW"][:20].tolist())

(array([1.34078635e-07]), array([[4.16945606e-16]]))

In [15]:
df["DOC"].tolist()[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]